<a href="https://colab.research.google.com/github/mlfa19/assignments/blob/master/Module%202/day04/Analyze%20Fake%20Coin%20Flips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequence Classification: Analyzing Coin Flips

In this notebook we'll be introducing a simple model of sequence classification based on the concept of n-grams.  You'll be following up more formally on these ideas in the assignment due next class.

> Data and some of the main ideas originally from [Can you Fake Coin Tosses?](https://faculty.math.illinois.edu/~hildebr/fakerandomness/)

## Modeling Sequences of Coin Flips

It turns out that humans are rather bad random number generators. If you ask a person to simulate a random sequence (e.g., flipping a coin a number of time), they will almost always introduce patterns in the sequence that you would not see if it were genuinely random.  A cool example of this in action is the [mind reader app](http://mindreaderpro.appspot.com/) put together by Yoav Freund's group at UC San Diego.

In this notebook we'll be using conditional probabilities as a way to understand the difference between real and fake coin toss sequences.

First, we'll load the data.

In [2]:
import pandas as pd
import gdown

gdown.download('https://drive.google.com/uc?authuser=0&id=1lAVLg9cxYxXL9kwIZI7T0xG128DVLlvw&export=download',
               'coin_flips.csv',
               quiet=False)
df_fake = pd.read_csv('coin_flips.csv')
df_fake

Downloading...
From: https://drive.google.com/uc?authuser=0&id=1lAVLg9cxYxXL9kwIZI7T0xG128DVLlvw&export=download
To: /content/coin_flips.csv
100%|██████████| 9.83k/9.83k [00:00<00:00, 5.75MB/s]


,student,flips
0,math199chp2017fall1,0000011001000011101010011101111010001110110100...
1,math199chp2017fall2,0010101100010111100101011000101100100011011001...
2,math199chp2017fall3,0000101010001010011111001011110010110000000101...
3,math199chp2017fall4,1101001110101001110101100001101101000100111010...
4,math199chp2017fall5,0010100111011011010110000110010011000101100110...
5,math199chp2017fall6,0010110001011110101011001101010001101011110010...
6,math199chp2017fall7,0001011010010111100101001001110110100100001101...
7,math199chp2017fall8,1011010011100101101000011101001110100110100110...
8,math199chp2017fall9,0011101100101001001011011000011010110111011101...
9,math199chp2017fall10,0010100011110101100100111001011001101000110101...


To get a better sense of one of these sequences, let's zoom in on the first entry.

In [3]:
print(df_fake.iloc[0]['flips'])
print('total flips', len(df_fake.iloc[0]['flips']))

0000011001000011101010011101111010001110110100111101000110010111100011111110011101100001011100010110001101000001010110100010100101000101101111101100101111010111010010111110010111001010101001011110010010000101
total flips 208


This shows a sequence of heads and tails (1 is heads and 0 is tails).  In this case there were a total of 208 flips.

## Classifying Sequences as Human or Computer Generated

While the pattern of heads (1's) and tails (0's) above might look random, it is reasonable to ask the question of whether there is some feature of the sequence that is different from an actual random sequence of coin flips.

As a way to test this we're going to generate a bunch of random sequences of coin flips using Python's `random` module (of course computers are only pseudo-random, but for our purposes here we can think of them as genuinely random).

In [0]:
from random import randint
def sample_random_flips(x):
    return ''.join(str(randint(0,1)) for _ in range(len(x)))

# df_real will be a data frame of random flips generated using python's random number
# generator.  Each sequence will have the same number of flips as one of the coin toss
# sequences generated by humans.
df_real = df_fake.copy()
df_real['flips'] = df_fake['flips'].map(sample_random_flips)

In the next block of code we're going to be creating some new columns in our data frames that represent the probability of seeing various binary sequences occur within either the real or fake data.  Once we've generated these columns, we'll include some more information on what the columns represent.

In [6]:
!pip install regex
import regex as re
import itertools

def count_overlapping(text, search_for):
    return len(re.findall(search_for, text, overlapped=True))

def make_seq_column(df, seq):
    df['seq_' + seq] = df['flips'].map(lambda x: count_overlapping(x, seq)/(len(x) - len(seq) + 1))
    
def populate_length_n_seqs(n):
    for s in itertools.product(*([['0', '1']]*n)):
        make_seq_column(df_real, ''.join(s))
        make_seq_column(df_fake, ''.join(s))

for n in range(10):
    populate_length_n_seqs(n)

     |████████████████████████████████| 645kB 2.8MB/s 


Let's look at the columns `seq_0` to see the probability of observing the length one sequence '0' in each of the sequences.  This represents the overall base rate of tails (0's) in each sequence.

In [7]:
df_fake['seq_0']

0     0.480769
1     0.533113
2     0.529412
3     0.500000
4     0.477987
5     0.493450
6     0.507692
7     0.489796
8     0.466019
9     0.505338
10    0.502646
11    0.465000
12    0.483568
13    0.477444
14    0.491892
15    0.487633
16    0.500000
17    0.503937
18    0.505000
19    0.450000
20    0.555000
21    0.585000
22    0.610000
23    0.550000
24    0.475000
25    0.440000
26    0.595000
27    0.500000
28    0.560000
29    0.520000
30    0.545000
31    0.510000
32    0.540000
33    0.510000
34    0.470000
35    0.510000
36    0.450000
37    0.530000
38    0.540000
39    0.485000
40    0.501538
Name: seq_0, dtype: float64

There are columns for each of the possible length 10 sequences.  For instance, we can check out `seq_01` to see the probability of observing the sequence "01" in each sequence.

In [8]:
df_fake['seq_01']

0     0.270531
1     0.279070
2     0.236686
3     0.306878
4     0.322785
5     0.337719
6     0.312741
7     0.302564
8     0.302439
9     0.307143
10    0.340426
11    0.296482
12    0.297170
13    0.290566
14    0.326087
15    0.262411
16    0.297561
17    0.292490
18    0.311558
19    0.376884
20    0.271357
21    0.211055
22    0.236181
23    0.336683
24    0.351759
25    0.346734
26    0.321608
27    0.326633
28    0.371859
29    0.346734
30    0.351759
31    0.190955
32    0.361809
33    0.381910
34    0.281407
35    0.336683
36    0.356784
37    0.361809
38    0.407035
39    0.306533
40    0.287037
Name: seq_01, dtype: float64

## Conditional Probabilities and the N-Gram Model

At first glance it looks like there are some differences across these sequences with regards to various sequences.  Is this enough to figure out whether a sequence is real or fake?

In order to figure this out we're going to use the concept of conditional probabilities.  Specifically, we'll be thinking about the probability of the next symbol in a sequence (in this case heads or tails) given the previous symbols in the sequence.

For instance, we might want to know what is the probability that the $n$th element of a sequence is 0 given that the previous element was a 1.  We can represent this as a conditional probability like so.

$$p(X_n=1|X_{n-1}=0)$$

This probability is also called a bigram since it looks at the probability related to a length 2 sequence (that is the sequence $X_{n-1}, X_{n}$.

If we allow for longer conditioning sequences, we get the $n$-gram model ($n$ is the total length of the sequence we're considering).  We'll leave the formal treatment of this for the assignment.

Next, we use the sequence columns we computed to define a function to compute conditional probabilities.  Instead of computing conditional probabilities for each sequence independently, we're going to look at the conditional probabilities over ***all the fake sequences***.

There is no need to get lost in the details of what is going on in this function.  All that is happening is applying the definition of conditional probability $p(A|B) = \frac{p(A,B)}{p(B)}$.

In [0]:
from functools import lru_cache

# Note: we are implicitly passing in df as a way to get around the fact that it is not hashable
@lru_cache(maxsize=10**5)
def get_conditional_probs(conditioning_seq):
    followed_by_1 = df['seq_' + conditioning_seq + '1'] * (df['flips'].map(len)-len(conditioning_seq)-1)
    followed_by_0 = df['seq_' + conditioning_seq + '0'] * (df['flips'].map(len)-len(conditioning_seq)-1)
    p_1_given_conditioning_seq = followed_by_1.sum()
    p_0_given_conditioning_seq = followed_by_0.sum()
    return p_0_given_conditioning_seq / (p_1_given_conditioning_seq + p_0_given_conditioning_seq), \
            p_1_given_conditioning_seq / (p_1_given_conditioning_seq + p_0_given_conditioning_seq)

In the next cell we're going to analyze the fake data and see what the conditional probability of the next symbol is given the conditioning symbol 010.

***Understanding Check***
What should the conditional probabilities be if the sequence is actually random?  What does the conditional probability you see below say about how humans generated random sequencdes.

In [10]:
# Please excuse passing in df_fake through a global variable.  This was a quick and dirty
# way of doing memoization with our get_conditional_probs function.
df = df_fake
get_conditional_probs('010')

(0.3364183139472274, 0.6635816860527727)

In order to see if our model can classify real versus fake sequences we're going to calculate the probability of a test sequence under two models.  The first model is the model derived from the fake sequences.  The second model always assigns a probability of 0.5 to each possible symbol (since for random sequences, each symbol should be independent of the previous symbol).

For the performance metric we're using the area under the ROC curve.  As we mentioned last class, the area under the ROC curve tells us the probability that we can correctly determine which of two points is of class 1 and which is of class 0 (assuming we are guaranteed there is exactly one of each).

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import time

def get_llrs(flips, context_length=2):
    """ Calculate the log likelihoood ratio of the data under the fake model versus the
        real model.  We use a log likelihood ratio for the same reason we used it in the
        Naive Bayes model.
        
        If you change context_length, you can see how the performance of the model varies
        as you use longer or shorter length sequences for your conditional probabilities.
        What seems to work the best? """
    llrs = []
    for s in flips:
        context = []
        llr = 0
        for flip in s:
            context_to_use = ''.join(context[-context_length:])
            p_0, p_1 = get_conditional_probs(context_to_use)
            if flip == '0':
                llr += np.log(p_0) - np.log(0.5)
            else:
                llr += np.log(p_1) - np.log(0.5)
            context.append(flip)
        llrs.append(llr)
    return llrs

all_roc_scores = []
context_length = 3
for iter in range(20):
    df_fake_train, df_fake_test = train_test_split(df_fake)
    get_conditional_probs.cache_clear()
    df = df_fake_train
    llrs_positive = np.array(get_llrs(df_fake_test['flips'], context_length))
    llrs_negative = np.array(get_llrs(df_real['flips'], context_length))
    all_llrs = np.concatenate((llrs_negative, llrs_positive))
    all_targets = np.concatenate((np.zeros(llrs_negative.shape), np.ones(llrs_positive.shape)))
    all_roc_scores.append(roc_auc_score(all_targets, all_llrs))

print("Accuracy averaged across 20 random runs is", np.array(all_roc_scores).mean())

Accuracy averaged across 20 random runs is 0.9039911308203992
